In [6]:
# %pip install python-telegram-bot transformers torch nest_asyncio

from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, filters, CallbackContext
import torch
import pickle
from transformers import (
    BertForSequenceClassification, 
    BertTokenizer, 
    T5ForConditionalGeneration,
    T5Tokenizer
)
import nest_asyncio
import asyncio

# Allow nested asyncio calls
nest_asyncio.apply()

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Intent Classification Model (BERT model in PyTorch)
intent_model = BertForSequenceClassification.from_pretrained('../model/intent_classifier_bert').to(device)
intent_tokenizer = BertTokenizer.from_pretrained('../model/bert_tokenizer')

# Load Response Generation Model (QA model using T5)
qa_model = T5ForConditionalGeneration.from_pretrained('../model/qa_model_t5').to(device)
qa_tokenizer = T5Tokenizer.from_pretrained('../model/t5_tokenizer')

# Load Greeting Response Model (for greeting responses)
greeting_model = BertForSequenceClassification.from_pretrained('../model/qa_greeting_model_mbert').to(device)
greeting_tokenizer = BertTokenizer.from_pretrained('../model/mbert_tokenizer')

# Load Label Encoder (for intent classification)
label_encoder = pickle.load(open('intentencoder.pkl', 'rb'))

# Predict Intent Function
def predict_intent(text):
    inputs = intent_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=32).to(device)
    with torch.no_grad():
        logits = intent_model(**inputs).logits
    intent_class = torch.argmax(logits, axis=1).item()
    return label_encoder.inverse_transform([intent_class])[0]

# Generate Answer Function (For QA)
def generate_answer(question):
    input_text = "question: " + question
    input_ids = qa_tokenizer.encode(input_text, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output_ids = qa_model.generate(input_ids)
    
    return qa_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Generate Greeting Response Function (For Greeting)
def generate_greeting_response(text):
    inputs = greeting_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=32).to(device)
    with torch.no_grad():
        logits = greeting_model(**inputs).logits
    greeting_response = torch.argmax(logits, axis=1).item()
    
    # Map the output to a response (You can add a predefined set of responses here)
    if greeting_response == 0:
        return "Hello! How can I assist you today?"
    else:
        return "Hi! I'm a bot created by a student at Cambodia Academy of Digital Technology to assist with Digital Landing in Cambodia."

# Predefined responses for simple queries
predefined_responses = {
    'hello': "Hi! How can I assist you today?",
    'goodbye': "Goodbye! Have a great day!",
    # Add more predefined responses if needed
}

# Start Command
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text("Hello! I am your QA bot. Ask me anything!")

# Handle Incoming Messages
async def handle_message(update: Update, context: CallbackContext):
    user_input = update.message.text.lower().strip()  # Normalize input

    # Check predefined responses
    if user_input in predefined_responses:
        await update.message.reply_text(predefined_responses[user_input])
        return
    
    # Predict intent
    intent = predict_intent(user_input)
    print(f"User Input: {user_input} | Predicted Intent: {intent}")  # Debugging
    
    # Check if the intent is "greeting"
    if intent == "greeting":
        # Use the Greeting model for dynamic greeting responses
        greeting_response = generate_greeting_response(user_input)
        await update.message.reply_text(greeting_response)
    elif intent == "banking":
        # If the intent is related to banking, generate an answer using the QA model
        answer = generate_answer(user_input)
        await update.message.reply_text(answer)
    else:
        # For out of domain or unknown intents
        await update.message.reply_text("Sorry, I couldn't understand that. Could you ask something else?")

# Main Function to Run Bot
def main():
    TOKEN = "8161447278:AAGCahgbJy3DAwY6XtTgcdIvbkr7DOJdPJc"  # Replace with your actual token
    app = ApplicationBuilder().token(TOKEN).build()
    
    # Add handlers
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    app.add_handler(MessageHandler(filters.COMMAND, start))  # Start command
    
    # Run bot
    print("🤖 Telegram bot is running...")
    app.run_polling()

if __name__ == '__main__':
    main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../model/qa_greeting_model_mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🤖 Telegram bot is running...
User Input: who are you | Predicted Intent:  greeting
User Input: who are you ? | Predicted Intent:  greeting
User Input: tell me about yourself | Predicted Intent:  greeting
User Input: what is digital lending? | Predicted Intent: out-of-domain


RuntimeError: Cannot close a running event loop